# [LRU Cache](https://leetcode.com/problems/lru-cache/description/)

Design a data structure to store key-value pairs with fast access and update. When capacity is exceeded, evict the least recently used item.
## Strategy
> Without capacity, this would be a simple hash, so you'll need another data structure. The O(1) requirement hints that additional structure must also have O(1) ability, like a linked list.
- Use a [doubly linked list](../resources/linked-list.ipynb) to keep track of usage order (most recent at head).
- Use a hash map for O(1) access to nodes.
- On get, move accessed node to head.
- On put, insert new node at head, remove tail if over capacity.

## Remember
- When removing a doubling linked list element with dummy nodes, you can use a generic code to remove any element


## Solution



In [ ]:
class Node {
  constructor(key?: number, val?: number) {
    this.key = key ?? null
    this.val = val ?? null
    this.next = null
    this.prev = null
  }
}

class LRUCache {
  constructor(capacity: number) {
      this.map = new Map<number, Node>()
      this.capacity = capacity
      this.head = new Node()
      this.tail = new Node()
      this.head.next = this.tail
      this.tail.prev = this.head
  }

  moveToFront(key: number) {
      const node: Node = this.map.get(key)

      // head node can remain where it is
      if(this.head.next !== node) {
        // remove node
        node.prev.next = node.next
        node.next.prev = node.prev
        
        node.next = this.head.next
        node.prev = this.head
        this.head.next.prev = node
        this.head.next = node
      }
  }

  get(key: number): number {
    if (this.map.has(key)) {
        this.moveToFront(key)
        return this.map.get(key).val
    }

    return -1
  }

  put(key: number, value: number): void {
    // just update if it already exists
    if (this.map.has(key)) {
      this.map.get(key).val = value
      this.moveToFront(key)
    } else {
      // remove the LRU
      if ((this.map.size + 1) > this.capacity) {
        // cull
        const lru: Node = this.tail.prev
        lru.prev.next = this.tail
        this.tail.prev = lru.prev

        // remove from the map
        this.map.delete(lru.key)
      }

      // store in front
      const node: Node = new Node(key, value)
      node.prev = this.head
      node.next = this.head.next
      this.head.next.prev = node
      this.head.next = node

      // store in map
      this.map.set(key, node)
    }
  }
}

## Test Cases



In [48]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("LRU Cache: put key 1", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  assertEquals(cache.get(1), 1);
});

Deno.test("LRU Cache: put key 2", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  assertEquals(cache.get(2), 2);
});

Deno.test("LRU Cache: evict least recently used key", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  cache.get(1);          // Access key 1
  cache.put(3, 3);       // Evicts key 2
  assertEquals(cache.get(2), -1);
});

Deno.test("LRU Cache: update value of existing key", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(1, 10);
  assertEquals(cache.get(1), 10);
});

Deno.test("LRU Cache: evict after capacity exceeded", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  cache.put(3, 3); // Evicts key 1
  assertEquals(cache.get(1), -1);
});

LRU Cache: put key 1 ... ok (1ms)
LRU Cache: put key 2 ... ok (0ms)
LRU Cache: evict least recently used key ... ok (0ms)
LRU Cache: update value of existing key ... ok (0ms)
LRU Cache: evict after capacity exceeded ... ok (0ms)

ok | 5 passed | 0 failed (2ms)
